<a href="https://colab.research.google.com/github/HappyJJins/Jeju_CreditCard/blob/master/submission0728.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, sys
from google.colab import drive

ROOT = '/content/drive'
drive.mount(ROOT)

my_path = '/content/notebooks'
# Colab Notebooks 안에 my_env 폴더에 패키지 저장
os.symlink('/content/drive/My Drive/Colab Notebooks/my_env', my_path)
sys.path.insert(0, my_path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
from google.colab import auth
auth.authenticate_user()

In [3]:
from google.cloud import bigquery
from tabulate import tabulate
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression

from itertools import product

# 평가 메트릭
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 통계
from scipy import stats
from scipy.stats import norm, skew #for some statistics

from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

In [4]:
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Jeju/data/jeju_data_ver1/201901-202003.csv')
sido_list = train['CARD_SIDO_NM'].unique().tolist()

In [10]:
for sido in sido_list:
  globals()['sub_{}'.format(sido)] = pd.read_json('/content/drive/My Drive/Colab Notebooks/Jeju/data/1903to2003_local/'+'{}.json'.format(sido), orient='table')

In [7]:
from numpy import inf
def sido_diff(sido):
  df = train[train['CARD_SIDO_NM'] == sido]
  df_19 = df[df['REG_YYMM'] == 201903]
  df_20 = df[df['REG_YYMM'] == 202003]
  diff = pd.concat([df_19.groupby('STD_CLSS_NM').AMT.sum(), df_20.groupby('STD_CLSS_NM').AMT.sum()], axis=1, keys = ['1903_AMT', '2003_AMT']).reset_index()
  diff = diff.fillna(0)
  diff['diff'] = (diff['1903_AMT']-diff['2003_AMT'])/diff['1903_AMT']
  diff['diff'] = diff['diff'].apply(lambda x: 0 if x == -inf else x)
  diff.columns = ['STD_CLSS_NM', '1903_AMT', '2003_AMT', 'diff']
  diff = diff[['STD_CLSS_NM', 'diff']]
  diff = diff.sort_values(by='diff', ascending=False).reset_index(drop=True)

  return diff

In [8]:
# 각 시도별 증감율 데이터프레임 생성
for sido in sido_list:
  tmp = sido_diff(sido)

  globals()['diff_{}'.format(sido)] = tmp

In [11]:
# 시도별로 뽑은 샘플로 예측한 df와 전체 데이터에서 2019년 03월과 2020년 03월을 뽑아 시도별로 증감율 계산한 df -> 하나로 병합
test_subs = [sub_강원, sub_경기, sub_경남, sub_경북, sub_광주, sub_대구, sub_대전, sub_부산, sub_서울, sub_세종, sub_울산, sub_인천, sub_전남, sub_전북, sub_제주, sub_충남, sub_충북]
test_diffs = [diff_강원, diff_경기, diff_경남, diff_경북, diff_광주, diff_대구, diff_대전, diff_부산, diff_서울, diff_세종, diff_울산, diff_인천, diff_전남, diff_전북, diff_제주, diff_충남, diff_충북]

In [14]:
# 혹시 몰라서 복사본 만듬
subs = test_subs.copy()
diffs = test_diffs.copy()

In [15]:
subs[0].info() # AMT <- float64 type

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72 entries, 0 to 71
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   REG_YYMM      72 non-null     int64  
 1   CARD_SIDO_NM  72 non-null     object 
 2   STD_CLSS_NM   72 non-null     object 
 3   AMT           72 non-null     float64
dtypes: float64(1), int64(1), object(2)
memory usage: 2.8+ KB


In [16]:
# 샘플로 예측한 값에서 증감율 고려해 수정
for i in range(len(sido_list)):
  subs[i] = subs[i].merge(diffs[i][['STD_CLSS_NM','diff']], how='left', on='STD_CLSS_NM')
  subs[i]['AMT'] = subs[i]['AMT']*(1-subs[i]['diff'])
  subs[i] = subs[i][['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM','AMT']]

In [17]:
subs[0].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72 entries, 0 to 71
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   REG_YYMM      72 non-null     int64  
 1   CARD_SIDO_NM  72 non-null     object 
 2   STD_CLSS_NM   72 non-null     object 
 3   AMT           72 non-null     float64
dtypes: float64(1), int64(1), object(2)
memory usage: 2.8+ KB


In [18]:
# 모든 시도 데이터 합치기
test = pd.concat(subs)
test = test.sort_values(by = ['REG_YYMM', 'CARD_SIDO_NM']).reset_index(drop = True)
test

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,강원,건강보조식품 소매업,2.271906e+08
1,202004,강원,골프장 운영업,4.116734e+08
2,202004,강원,과실 및 채소 소매업,7.117371e+07
3,202004,강원,관광 민예품 및 선물용품 소매업,2.322025e+07
4,202004,강원,그외 기타 스포츠시설 운영업,1.424509e+07
...,...,...,...,...
1207,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,2.387591e+07
1208,202007,충북,한식 음식점업,8.432970e+07
1209,202007,충북,호텔업,2.166644e+07
1210,202007,충북,화장품 및 방향제 소매업,5.491836e+07


In [19]:
project_id = 'jeju-282707'
client = bigquery.Client(project = 'jeju-282707')
submission = client.query('''
  SELECT 
    * 
  FROM `jeju-282707.jeju_data_ver1.submission` 
  ''').to_dataframe()

In [37]:
# 제출파일 양식에 맞게 가공
final = submission.drop(['AMT'], axis=1)
final = final.merge(test, on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
final

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,0,202004,강원,건강보조식품 소매업,2.271906e+08
1,1,202004,강원,골프장 운영업,4.116734e+08
2,2,202004,강원,과실 및 채소 소매업,7.117371e+07
3,3,202004,강원,관광 민예품 및 선물용품 소매업,2.322025e+07
4,4,202004,강원,그외 기타 분류안된 오락관련 서비스업,NaN
...,...,...,...,...,...
1389,1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,2.387591e+07
1390,1390,202007,충북,한식 음식점업,8.432970e+07
1391,1391,202007,충북,호텔업,2.166644e+07
1392,1392,202007,충북,화장품 및 방향제 소매업,5.491836e+07


In [32]:
final_nan = final[final['AMT'].isnull()].reset_index(drop=True)
final_nan

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,4,202004,강원,그외 기타 분류안된 오락관련 서비스업,NaN
1,14,202004,강원,면세점,NaN
2,28,202004,강원,자동차 임대업,NaN
3,30,202004,강원,정기 항공 운송업,NaN
4,35,202004,강원,택시 운송업,NaN
...,...,...,...,...,...
181,1357,202007,충북,그외 기타 분류안된 오락관련 서비스업,NaN
182,1368,202007,충북,버스 운송업,NaN
183,1376,202007,충북,여행사업,NaN
184,1381,202007,충북,자동차 임대업,NaN


In [22]:
train_03 = train[train['REG_YYMM']==202003]
train_03.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1260508 entries, 23437284 to 24697791
Data columns (total 12 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   REG_YYMM      1260508 non-null  int64 
 1   CARD_SIDO_NM  1260508 non-null  object
 2   CARD_CCG_NM   1255817 non-null  object
 3   STD_CLSS_NM   1260508 non-null  object
 4   HOM_SIDO_NM   1260508 non-null  object
 5   HOM_CCG_NM    1253658 non-null  object
 6   AGE           1260508 non-null  object
 7   SEX_CTGO_CD   1260508 non-null  int64 
 8   FLC           1260508 non-null  int64 
 9   CSTMR_CNT     1260508 non-null  int64 
 10  AMT           1260508 non-null  int64 
 11  CNT           1260508 non-null  int64 
dtypes: int64(6), object(6)
memory usage: 125.0+ MB


In [25]:
# final_nan = final_nan.drop(['AMT'], axis=1)
amt_nan = []
for idx, x in final_nan.iterrows():
  amt = sum(train_03[(train_03['CARD_SIDO_NM']==x['CARD_SIDO_NM']) & (train_03['STD_CLSS_NM']==x['STD_CLSS_NM'])]['AMT'])
  amt_nan.append(amt)

In [27]:
len(amt_nan)

186

In [33]:
amt_nan = pd.Series(amt_nan)
final_nan['AMT'] = amt_nan

In [34]:
final_nan

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0
1,14,202004,강원,면세점,0
2,28,202004,강원,자동차 임대업,4607000
3,30,202004,강원,정기 항공 운송업,0
4,35,202004,강원,택시 운송업,0
...,...,...,...,...,...
181,1357,202007,충북,그외 기타 분류안된 오락관련 서비스업,0
182,1368,202007,충북,버스 운송업,0
183,1376,202007,충북,여행사업,0
184,1381,202007,충북,자동차 임대업,0


In [35]:
final_nan[final_nan['AMT'] != 0]

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
2,28,202004,강원,자동차 임대업,4607000
6,53,202004,경기,내항 여객 운송업,261500
9,86,202004,경남,그외 기타 분류안된 오락관련 서비스업,482700
11,105,202004,경남,여행사업,36000
16,146,202004,경북,여행사업,171000
17,151,202004,경북,자동차 임대업,778000
27,213,202004,대구,기타 수상오락 서비스업,72000
32,254,202004,대전,기타 수상오락 서비스업,94000
37,274,202004,대전,자동차 임대업,1276500
66,474,202004,인천,여행사업,1580000


In [40]:
final = final.dropna(subset=['AMT'])
final

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,0,202004,강원,건강보조식품 소매업,2.271906e+08
1,1,202004,강원,골프장 운영업,4.116734e+08
2,2,202004,강원,과실 및 채소 소매업,7.117371e+07
3,3,202004,강원,관광 민예품 및 선물용품 소매업,2.322025e+07
5,5,202004,강원,그외 기타 스포츠시설 운영업,1.424509e+07
...,...,...,...,...,...
1389,1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,2.387591e+07
1390,1390,202007,충북,한식 음식점업,8.432970e+07
1391,1391,202007,충북,호텔업,2.166644e+07
1392,1392,202007,충북,화장품 및 방향제 소매업,5.491836e+07


In [44]:
final = pd.concat([final, final_nan])
final = final.sort_values('id').reset_index(drop=True)

In [45]:
final

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,0,202004,강원,건강보조식품 소매업,2.271906e+08
1,1,202004,강원,골프장 운영업,4.116734e+08
2,2,202004,강원,과실 및 채소 소매업,7.117371e+07
3,3,202004,강원,관광 민예품 및 선물용품 소매업,2.322025e+07
4,4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0.000000e+00
...,...,...,...,...,...
1389,1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,2.387591e+07
1390,1390,202007,충북,한식 음식점업,8.432970e+07
1391,1391,202007,충북,호텔업,2.166644e+07
1392,1392,202007,충북,화장품 및 방향제 소매업,5.491836e+07


In [47]:
final['AMT'] = final['AMT'].astype(int)
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1394 entries, 0 to 1393
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1394 non-null   int64 
 1   REG_YYMM      1394 non-null   int64 
 2   CARD_SIDO_NM  1394 non-null   object
 3   STD_CLSS_NM   1394 non-null   object
 4   AMT           1394 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 54.6+ KB


In [51]:
final.to_csv('/content/drive/My Drive/Colab Notebooks/Jeju/submission.csv', encoding='utf-8-sig', index=False)

점수 많이 올랐다ㅠㅠㅠㅠㅠㅠ